In [1]:
import matplotlib.pyplot as plt
import numpy as np
import skimage
from torch.utils.data import Dataset 
import os
from IPython import display
import torch
import torch.nn as nn
from skimage import io
from torchvision import transforms
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from datetime import datetime
%matplotlib inline
import torchvision
from PIL.Image import open, BILINEAR
import os
import random
from PIL import Image
import random

In [312]:
import cv2
vidcap = cv2.VideoCapture('heatmaps3500.mp4')
success,image = vidcap.read()
count = 0
while success:
    cv2.imwrite("./heatmap/%d.jpg" % count, image)     # save frame as JPEG file      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  False


In [313]:
def sort_names(img_name):
    number = int(img_name[:-4])
    return number
    

In [348]:
class my_dataset(Dataset):
    def __init__(self, img_dir, transform=None, start=3000, stop=4500):
        super(my_dataset, self).__init__()
        
        if stop>0:
            imgs_names = sorted(next(os.walk(img_dir))[2], key=sort_names)[start:stop]
        else:
            imgs_names = sorted(next(os.walk(img_dir))[2], key=sort_names)
            
        self.img = [img_dir+'/'+ i for i in imgs_names]  
        self.transform = transform
        
    def __len__(self):
        return len(self.img)
    
    def __getitem__(self, idx):
        image_path = self.img[idx]
        img = Image.open(image_path)
        if self.transform != None:
            img = self.transform(img)    
        return img



In [349]:
transform = transforms.Compose([
        transforms.Resize((352, 512),  interpolation = 0),
        transforms.ToTensor(),
        ])

In [341]:
# import glob

In [342]:
# glob.glob(os.path.join('./video_youtube', '*.jpg'))

In [350]:
batch_size = 1

In [351]:
dataset= my_dataset('./video_youtube', transform)
dataset_heat= my_dataset('./heatmap', transform, stop=0)

In [352]:
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=False)
dataloader_heat = DataLoader(dataset_heat, batch_size = batch_size, shuffle=False)

In [353]:
ex= next(iter(dataloader))
ex= next(iter(dataloader_heat))

In [354]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device =torch.device("cpu")

In [355]:
class UnetDownBlock(nn.Module):
    def __init__(self, in_channels, out_channels, pooling=True):
        super().__init__()

        self.convs = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
    def forward(self, x):

        out_before_pooling = self.convs(x)
        out = self.maxpool(out_before_pooling)

        return out, out_before_pooling
      

In [356]:
class UnetUpBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        # your code here
        self.upsample = nn.Upsample(scale_factor=2)
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels * 2, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        
    def forward(self, x, x_bridge):
        x_up = self.upsample(x)
        x_concat = torch.cat([x_up, x_bridge], dim=1)
        out = self.convs(x_concat)
        
        return out

In [357]:

class Unet(nn.Module):
    def __init__(self, n_base_channels=64):
        super().__init__()
        
        # your code here
        self.down_blocks = nn.ModuleList([
            UnetDownBlock(3, n_base_channels),
            UnetDownBlock(n_base_channels, n_base_channels * 2),
            UnetDownBlock(n_base_channels * 2, n_base_channels * 4),
            UnetDownBlock(n_base_channels * 4, n_base_channels * 4),
            UnetDownBlock(n_base_channels * 4, n_base_channels * 4)
        ])
        self.up_blocks = nn.ModuleList([
            UnetUpBlock(n_base_channels * 4, n_base_channels * 4),
            UnetUpBlock(n_base_channels * 4, n_base_channels * 2),
            UnetUpBlock(n_base_channels * 2, n_base_channels),
            UnetUpBlock(n_base_channels, n_base_channels),
        ])
        self.final_block = nn.Sequential(
            nn.Conv2d(n_base_channels, 3, kernel_size=3, padding=1),
        )
            
        
    def forward(self, x):
        
        # your code here
        out = x
        outputs_before_pooling = []
        for i, block in enumerate(self.down_blocks):
            out, before_pooling = block(out)
            outputs_before_pooling.append(before_pooling)
        out = before_pooling
        
        for i, block in enumerate(self.up_blocks):    
            out = block(out, outputs_before_pooling[-i - 2])
        out = self.final_block(out)
        
        return out

In [358]:
net = Unet(n_base_channels=64).to(device)
net2 = Unet(n_base_channels=64).to(device) 

In [359]:
net.load_state_dict(torch.load('./unet_64_cosine_n.pth', map_location=device))
# net2.load_state_dict(torch.load('./unet_64_cosine_n.pth', map_location=device))
# net2.load_state_dict(torch.load('./unet_64_cosine_Jitter.pth', map_location=device))
# net2.load_state_dict(torch.load('./unet_64_diff_lr_n.pth', map_location=device))
net2.load_state_dict(torch.load('./unet_64_sgd_n.pth', map_location=device))
net.eval()
net2.eval()
print()

In [370]:
frames = []
with torch.no_grad():
    for image, heat in zip(dataloader, dataloader_heat):
        image = image.to(device)
        pred = net(image)
        
        heat = heat.squeeze(0).permute(1,2,0).cpu().detach().numpy()*255
        image_out = image.squeeze(0).permute(1,2,0).cpu().detach().numpy()*255
        pred_out = torch.argmax(pred, dim=1).squeeze(0).detach().cpu().numpy()*(255/2)
        
        image_out_with=image_out*0.4+pred_out[..., np.newaxis]*0.6
        
        image_out = image_out.astype(np.uint8)
        pred_out = pred_out.astype(np.uint8)
        image_out_with = image_out_with.astype(np.uint8)
#         zero = np.ones_like(image_out)*200
        zero = heat.astype(np.uint8)

        
        a = np.concatenate((image_out, image_out_with), axis=1)
        pred = np.concatenate((pred_out[..., np.newaxis], pred_out[..., np.newaxis], pred_out[..., np.newaxis]), axis=2)
        b = np.concatenate((zero, pred), axis=1)
        
        out = np.concatenate((a, b), axis=0)

#         plt.figure(figsize=(20,15))
#         plt.imshow(Image.fromarray(out))
#         plt.show()
        frames.append(out)


In [371]:
frames[0].shape

(704, 1024, 3)

In [372]:
frames_bgr = [rgb[...,::-1] for rgb in frames]

In [373]:
frames_bgr[0].shape

(704, 1024, 3)

In [375]:
import cv2
import numpy as np


height,width,layers=frames_bgr[0].shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video=cv2.VideoWriter('full_video_speeded.mp4',fourcc,48,(width,height))

for im in frames_bgr:
    video.write(im)

cv2.destroyAllWindows()
video.release()

In [311]:
# files = []
with torch.no_grad():
    for idx, image in enumerate(dataloader):
        image = image.to(device)
        pred = net(image)
        pred_2 = net2(image)
        
        image_out = image.squeeze(0).permute(1,2,0).cpu().detach().numpy()*255
        pred_out = torch.argmax(pred, dim=1).squeeze(0).detach().cpu().numpy()*(255/2)
        pred_out_2 = torch.argmax(pred_2, dim=1).squeeze(0).detach().cpu().numpy()*(255/2)
        
        image_out+=pred_out[..., np.newaxis]
        image_out = image_out.astype(np.uint8)
        pred_out = pred_out.astype(np.uint8)
        pred_out_2 = pred_out_2.astype(np.uint8)
#         files.append(pred_out)
#         print(idx, np.unique(pred_out), np.unique(pred_out_2))
        plt.figure(figsize=(20,15))
        plt.subplot(1,3,1)
        plt.imshow(Image.fromarray(image_out))
        plt.subplot(1,3,2)
        plt.imshow(Image.fromarray(pred_out))
        plt.subplot(1,3,3)
        plt.imshow(Image.fromarray(pred_out_2))
        plt.show()
#         if idx>100:
#             break 